In [40]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import shap

np.random.seed(42)

In [41]:
years = np.array(range(2012, 2024))
features = ["18 to 64 years poverty [%]", "18 years and over [%]",
    "65 years and over [%]", "65 years and over poverty [%]",
    "All parents in family in labor force [%]",
    "American Indian and Alaska Native [%]", "Armed Forces [%]",
    "Asian [%]", "Black or African American [%]",
    "Civilian labor force [%]", "Commuting: individual car [%]",
    "Commuting: public transport [%]", "Commuting: work from home [%]",
    "Education: 9th-12th (no diploma) [%]", "Education: <9th grade [%]",
    "Education: associate's degree [%]", "Education: bachelor's degree [%]",
    "Education: bachelor's degree or higher [%]",
    "Education: graduate or professional degree [%]",
    "Education: high school graduate [%]",
    "Education: high school graduate or higher [%]",
    "Education: some college (no degree) [%]",
    "Female civilian labor force [%]", "Foreign-born [%]",
    "Foreign-born: naturalized [%]", "Gross rent <25% of income [%]",
    "Gross rent >=25% of income [%]",
    "Health insurance coverage (noninstitutionalized) [%]",
    "Hispanic or Latino [%]", "Homeowner vacancy rate [%]",
    "Households: Female (no spouse) [%]",
    "Households: Male (no spouse) [%]", "Households: married-couple [%]",
    "Households: with broadband [%]", "Households: with computer [%]",
    "Housing units with a mortgage [%]",
    "Labor force by industry: agriculture, forestry, fishing, hunting, mining [%]",
    "Labor force by industry: arts, entertainment, recreation, accommodation, food services [%]",
    "Labor force by industry: construction [%]",
    "Labor force by industry: education, health care, social assistance [%]",
    "Labor force by industry: finance, insurance, real estate, rental, leasing [%]",
    "Labor force by industry: information [%]",
    "Labor force by industry: manufacturing [%]",
    "Labor force by industry: other services [%]",
    "Labor force by industry: professional, scientific, management, admin, waste management services [%]",
    "Labor force by industry: public administration [%]",
    "Labor force by industry: retail trade [%]",
    "Labor force by industry: transportation, warehousing, utilities [%]",
    "Labor force by industry: wholesale trade [%]", "Land area (km²)",
    "Limited English speakers [%]", "Male [%]",
    "Mean travel time to work (min)", "Median age",
    "Median full-time earnings gender ratio (F/M) [%]",
    "Median household income ($)", "Moved: different county [%]",
    "Moved: different house [%]", "Moved: different state [%]",
    "Moved: within U.S. [%]",
    "Owner costs <25% of income (with mortgage) [%]",
    "Owner costs <25% of income (without mortgage) [%]",
    "Owner costs >=25% of income (with mortgage) [%]",
    "Owner costs >=25% of income (without mortgage) [%]",
    "Rental vacancy rate [%]", "Total households", "Total housing units",
    "Total population", "Under 18 years poverty [%]",
    "Unemployment Rate [%]", "Vacant housing units [%]", "Water area (km²)",
    "White [%]"]

In [42]:
class RNN_Regressor(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        lstm_hidden_size = 32
        hidden_size = 4

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=lstm_hidden_size, batch_first=True)

        self.dense1 = nn.Linear(lstm_hidden_size, hidden_size)
        self.dense2 = nn.Linear(hidden_size, output_size)

        

    def forward(self, input):
        x = F.dropout(input, p=0.35, training=self.training)

        x, _ = self.lstm(x)
        x = x[:, -1, :]

        x = self.dense1(x)
        x = F.leaky_relu(x)

        x = self.dense2(x)
        x = F.sigmoid(x)

        return x

In [43]:
def data_by_year(dict_x):
    all_keys = list(dict_x.keys())
    data_by_year_x = []

    for y in range(len(years)):
        data_year_x = []
        for id in all_keys:
            data_year_x.append(dict_x[id][y])
        tensor_x = torch.tensor(np.stack(data_year_x), dtype=torch.float32)
        data_by_year_x.append(tensor_x)

    print(len(data_by_year_x), data_by_year_x[0].shape)

    return data_by_year_x

In [44]:
loaded = np.load("../../Data/Data/data_x.npz", allow_pickle=True)
dict_x = {key: loaded[key] for key in loaded.files}

data_by_year_x = data_by_year(dict_x)

model = RNN_Regressor(data_by_year_x[0].shape[2], 1)
incompat  = model.load_state_dict(torch.load("../../Model/Model/model.pt"))
model.eval()

12 torch.Size([3096, 3, 73])


RNN_Regressor(
  (lstm): LSTM(73, 32, batch_first=True)
  (dense1): Linear(in_features=32, out_features=4, bias=True)
  (dense2): Linear(in_features=4, out_features=1, bias=True)
)

In [45]:
all_data_x = torch.tensor(np.array([arr for sublist in dict_x.values() for arr in sublist]), dtype=torch.float32)

background_size = 1
idx = np.random.choice(all_data_x.shape[0], size=background_size, replace=False)
background = all_data_x[idx]

explainer = shap.GradientExplainer(model, background)

In [46]:
def sensitivity_analysis(data_by_x):
    shap_values_list = []
    i = 1

    for data_x in data_by_x:
        shap_values = explainer.shap_values(data_x, rseed=42)
        shap_values_list.append(shap_values.squeeze(axis=-1))
        print(f"Processing item {i}/{len(data_by_x)} ({(i / len(data_by_x)) * 100:.1f}%)")
        i += 1

    return np.array(shap_values_list)

In [47]:
shap_by_year = sensitivity_analysis(data_by_year_x[:2])

Processing item 1/2 (50.0%)
Processing item 2/2 (100.0%)


In [49]:
shap_by_year.shape

(2, 3096, 3, 73)